# Feel like Alessandro is gonna hate on me for using a jupyter but

## Dependencies

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm

import pandas as pd
import json

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

import ast

import sqlite3

from storage import Storage	

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
storage = Storage()
device

import os
print(os.getcwd())

data.db
c:\Users\20221737\Documents\Honors AI\spoticolor-main\spoticolor


## Data

### Loading

In [3]:
# store = storage.Storage()
validTracks = storage.getValidTracks()
print(validTracks)

[]


In [4]:
tracks_data = []

for track_id, track_data in validTracks:
    try:
       decoded_data = json.loads(track_data)
       tracks_data.append((track_id, decoded_data))
    except json.JSONDecodeError:
        print(f"Error processing data for track_id: {track_id}. Data: {track_data}")

print(tracks_data)

In [6]:
tracks_data[0][1]['segments'][1]

IndexError: list index out of range

In [ ]:
# Broken track
store.getTrack('05wb6MEyJeG2z1MVCIXxC8')

### Preprocessing

In [ ]:
features = ['start', 'duration', 'confidence', 'loudness_start', 'loudness_max', 'loudness_max_time', 'loudness_end', 'pitches', 'timbre']

scalers = {feature: MinMaxScaler() for feature in ['start', 'duration', 'confidence', 'loudness_start']}
# Feel like min max is worse for loudenss but idk we have to test it ig
scalers.update({feature: StandardScaler() for feature in ['loudness_start', 'loudness_max', 'loudness_end']})

for feature in features:
    all_values = np.array([seg[feature] for _, track_data in tracks_data for seg in track_data['segments']]).reshape(-1, 1)
    scalers[feature].fit(all_values)
    
    for _, track_data in tracks_data:
        for seg in track_data['segments']:
            seg[feature] = scalers[feature].transform([[seg[feature]]])[0][0]

In [ ]:
tracks_data[0][1]['segments'][1]

In [ ]:
def flatten_segments(segments):
    flattened_segments = [np.array([
        seg['start'], seg['duration'], seg['confidence'], seg['loudness_start'], 
        seg['loudness_max_time'], seg['loudness_max'], seg['loudness_end']
    ] + seg['pitches'] + seg['timbre']) for seg in segments]
    
    return np.array(flattened_segments)

def pad_tracks(tracks, max_segments, segment_size=31):
    padded_tracks = np.zeros((len(tracks), max_segments, segment_size))
    
    for i, (_, track_data) in enumerate(tracks):
        flattened_segments = flatten_segments(track_data['segments'])
        num_segments = len(flattened_segments)
        padded_tracks[i, :num_segments, :] = flattened_segments[:max_segments]
    
    return padded_tracks

max_segments = max(len(track_data['segments']) for _, track_data in tracks_data)

padded_tracks = pad_tracks(tracks_data, max_segments)

In [ ]:
padded_tracks.shape # (n_tracks, n_segments, n_features)

In [ ]:
paddedTracks_train, paddedTracks_test = train_test_split(padded_tracks, test_size=0.2, random_state=42)

tracksTensor_train = torch.tensor(paddedTracks_train, dtype=torch.float32).to(device)
trainDataset = TensorDataset(tracksTensor_train)
trainDataloader = DataLoader(trainDataset, batch_size=32, shuffle=True)

tracksTensor_test = torch.tensor(paddedTracks_test, dtype=torch.float32).to(device)
testDataset = TensorDataset(tracksTensor_test)
testDataloader = DataLoader(testDataset, batch_size=32, shuffle=False) 

## Model

### Definition

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim=31, encoding_dim=3):
        super(Model, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(True),
            nn.BatchNorm1d(64),
            nn.Dropout(0.1),
            nn.Linear(64, 32),
            nn.ReLU(True),
            nn.BatchNorm1d(32),
            nn.Linear(32, encoding_dim),
            nn.ReLU(True)
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 32),
            nn.ReLU(True),
            nn.BatchNorm1d(32),
            nn.Linear(32, 64),
            nn.ReLU(True),
            nn.BatchNorm1d(64),
            nn.Dropout(0.1),
            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        return encoded, decoded
    
    def encode(self, x):
        return self.encoder(x)

### Training

In [ ]:
model = Model(input_dim=31, encoding_dim=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss()

epochs = 100

for epoch in range(epochs):
    model.train() 
    running_loss = 0.0

    with tqdm(trainDataloader, unit="batch", desc=f"Epoch {epoch + 1}/{epochs}") as tepoch:
        for batch in tepoch:
            inputs = batch[0].to(device)
            inputs = inputs.view(-1, 31) 

            optimizer.zero_grad()

            _, decoded = model(inputs)
            loss = criterion(decoded, inputs)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            tepoch.set_postfix(loss=(running_loss / (tepoch.n + 1)))


            epoch_loss = running_loss / len(trainDataloader)

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}')
